<div class="alert alert-block alert-warning">
    Purpose: Use dataset generated by <a href="https://github.com/orbisgis/geoclimate">geoclimate</a> inside the administrative boundaries to create a map with the following layers: 
        <ol>
        <li>
        Administrative boundary.
        </li>
        <li>
        Buildings with and without height information on OSM.
        </li>
        <li>
        Building categorized by number of levels.
        </li>
        <li>
        Building categorized by height.
        </li>
        <li>    
        Building main use.
            </li>
        </ol>
</div>

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
import matplotlib.colors as colors

## Administrative boundaries - Halle, Germany

In [ ]:
center = (51.473334, 11.966667)

In [ ]:
map = folium.Map(center, min_zoom=12, max_bounds=True, prefer_canvas=True)

In [ ]:
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    overlay = False,
    control = True
).add_to(map)

In [ ]:
## Retrieving admninistrative boundaries
# See: https://peteris.rocks/blog/openstreetmap-administrative-boundaries-in-geojson/

In [ ]:
# map

In [ ]:
admbnd = folium.GeoJson('halle.geojson', name='Boundary', style_function = lambda x: {'fillColor': 'none', 'stroke': 'false', 'color': 'gray'})
admbnd.add_to(map)

## Buildings

In [ ]:
buildings = gpd.read_file('building_halle.geojson')

In [ ]:
buildings.head()

In [ ]:
totbld = len(buildings.index)

In [ ]:
print(f'Total buildings in OSM map: {totbld}')

### OSM buildings missing height information

In [ ]:
bld_hgt_miss = pd.read_csv('building_height_missing.csv')

In [ ]:
bld_hgt_miss.head()

In [ ]:
missing_list = bld_hgt_miss.ID_SOURCE.values.tolist()

In [ ]:
buildings['osm_height'] = np.where(buildings['ID_SOURCE'].isin(missing_list), 'missing', 'available')

In [ ]:
missing = buildings[buildings.osm_height == 'missing']
available = buildings[buildings.osm_height == 'available']

In [ ]:
print(f'Total buildings missing height: {len(missing)}')

In [ ]:
buildings.ID_BUILD.count(), missing.ID_BUILD.count(), available.ID_BUILD.count()

# Summary

In [ ]:
buildings.HEIGHT_WALL.describe()

In [ ]:
buildings.HEIGHT_WALL[buildings.osm_height=='available'].describe()

## Mapping

In [ ]:
map = buildings.explore(m=map, column='osm_height', cmap=colors.ListedColormap(['#D94325','#5CD925']), name='Buildings')

In [ ]:
map = buildings.explore(m=map, column='NB_LEV', name='Levels')

In [ ]:
map = buildings.explore(m=map, column='HEIGHT_WALL', name='Height')

In [ ]:
map = buildings.explore(m=map, column='MAIN_USE', name='Main use')

In [ ]:
folium.LayerControl().add_to(map)

In [ ]:
map

In [ ]:
map.save('building_height_data.html')